In [28]:
## Each Space is a cell in a juptyr notebook ("QuickThinking" in fastai_workbench) that runs on Gradient Paperspace
from greek_normalisation.utils import (nfd, nfc, nfkc, convert_to_2019)
import numpy as np
import datetime
# from gensim.models import Phrases
from gensim.models import Word2Vec
from gensim.corpora import Dictionary

In [29]:
import pickle
import bz2
from greek_normalisation.utils import (nfd, nfc, nfkc, convert_to_2019)

def deserialize(mode_of_dataset):
    word2vec_documents_file = \
        bz2.BZ2File(f'{base_path}/{mode_of_dataset}_word2vec_documents_chrysostom.pickle', 'rb')
    lda_documents_file = \
        bz2.BZ2File(f'{base_path}/{mode_of_dataset}_lda_documents_chrysostom.pickle', 'rb')
    lda_dictionary_file = \
        bz2.BZ2File(f'{base_path}/{mode_of_dataset}_lda_dictionary_chrysostom.pickle', 'rb')
    lda_corpus_file = \
        bz2.BZ2File(f'{base_path}/{mode_of_dataset}_lda_corpus_chrysostom.pickle', 'rb')

    word2vec_documents = pickle.load(word2vec_documents_file)
    lda_documents = pickle.load(lda_documents_file)
    lda_dictionary = pickle.load(lda_dictionary_file)
    lda_corpus = pickle.load(lda_corpus_file)

    word2vec_documents_file.close()
    lda_documents_file.close()
    lda_dictionary_file.close()
    lda_corpus_file.close()

    return [word2vec_documents, lda_documents, lda_dictionary, lda_corpus]


base_path = '/Users/paul/dev/fastai_workbench'
path = f'{base_path}/ALL_REAL_JOHN_FOR_BAG_OF_WORDS.TXT'
mode_of_dataset = "development"
word2vec_documents, lda_documents, lda_dictionary, lda_corpus = \
    deserialize(mode_of_dataset)
# print(word2vec_documents )



In [30]:
#Train Word2Vec model
#https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-download-auto-examples-tutorials-run-word2vec-py

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.models import Word2Vec

word2vec_model = Word2Vec(
    sentences=word2vec_documents,
    min_count=10,
    size=200,
    workers=4, #requires Cython to be installed to have any effect. Othewise you are stuck on 1 core.
)


2021-02-07 00:34:07,167 : INFO : collecting all words and their counts
2021-02-07 00:34:07,169 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-07 00:34:07,180 : INFO : collected 8218 word types from a corpus of 61363 raw words and 1106 sentences
2021-02-07 00:34:07,181 : INFO : Loading a fresh vocabulary
2021-02-07 00:34:07,186 : INFO : effective_min_count=10 retains 1262 unique words (15% of original 8218, drops 6956)
2021-02-07 00:34:07,187 : INFO : effective_min_count=10 leaves 44679 word corpus (72% of original 61363, drops 16684)
2021-02-07 00:34:07,192 : INFO : deleting the raw counts dictionary of 8218 items
2021-02-07 00:34:07,193 : INFO : sample=0.001 downsamples 58 most-common words
2021-02-07 00:34:07,194 : INFO : downsampling leaves estimated 40489 word corpus (90.6% of prior 44679)
2021-02-07 00:34:07,197 : INFO : estimated required memory for 1262 words and 200 dimensions: 2650200 bytes
2021-02-07 00:34:07,197 : INFO : resetting layer we

In [31]:
print(word2vec_model.most_similar_cosmul(nfc('λόγος')))

<ipython-input-31-50fa1339bfb7>:1: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  print(word2vec_model.most_similar_cosmul(nfc('λόγος')))
2021-02-07 00:34:13,589 : INFO : precomputing L2-norms of word weight vectors


[('ἔρχομαι', 0.9999788999557495), ('κόσμος', 0.9999776482582092), ('πόλις', 0.9999774694442749), ('μυρίος', 0.9999773502349854), ('πρᾶγμα', 0.9999772310256958), ('καιρός', 0.9999768137931824), ('δείκνυμι', 0.9999767541885376), ('ἔργον', 0.9999767541885376), ('πρῶτος', 0.999976634979248), ('βούλομαι', 0.9999765157699585)]


In [32]:
# Train LDA model.
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.models import LdaModel

# Set training parameters.
num_topics = 50 #15
chunksize = 1500
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = lda_dictionary[0]  # This is only to "load" the lda_dictionary.
id2word = lda_dictionary.id2token

model = LdaModel(
    corpus = lda_corpus,
    id2word = id2word,
    chunksize = chunksize,
    alpha = 'auto',
    eta = 'auto',
    iterations = iterations,
    num_topics = num_topics,
    passes = passes,
    eval_every = eval_every
)

top_topics = model.top_topics(lda_corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)
from pprint import pprint
pprint(top_topics)

2021-02-07 00:35:02,213 : INFO : using autotuned alpha, starting with [0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
2021-02-07 00:35:02,214 : INFO : using serial LDA version on this node
2021-02-07 00:35:02,217 : INFO : running online (multi-pass) LDA training, 50 topics, 20 passes over the supplied corpus of 1106 documents, updating model once every 1106 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2021-02-07 00:35:02,218 : INFO : PROGRESS: pass 0, at document #1106/1106
2021-02-07 00:35:03,444 : INFO : optimized alpha [0.01863195, 0.01859252, 0.018846061, 0.019274587, 0.01890413, 0.01884613, 0.018495375, 0.018572297, 0.01855404, 0.018709118, 0.018534312, 0.01872937, 0.018

Average topic coherence: -2.9927.
[([(0.11858796, 'δέω1'),
   (0.043180685, 'λύω'),
   (0.040224496, 'δεσμός'),
   (0.028007697, 'δεσμεύω'),
   (0.021980744, 'ἅλυσις'),
   (0.020628912, 'παῦλος'),
   (0.019862356, 'πάσχω'),
   (0.01985985, 'δεσμὰ'),
   (0.017132629, 'ποιέω'),
   (0.01697656, 'ἀκούω'),
   (0.015953414, 'δεσμωτήριον'),
   (0.015745383, 'χριστός'),
   (0.013666418, 'ἀνοίγνυμι'),
   (0.013269463, 'χριστὸν'),
   (0.012792043, 'ἐργάζομαι'),
   (0.01227566, 'πρᾶγμα'),
   (0.011379438, 'πράσσω'),
   (0.010688683, 'μέλλω'),
   (0.010292429, 'δόξα'),
   (0.010050446, 'μακάριος')],
  -1.8358144621197954),
 ([(0.08535116, 'χριστός'),
   (0.06561399, 'ἰησοῦς'),
   (0.048922956, 'κύριος'),
   (0.03813919, 'ἅγιος'),
   (0.033320718, 'χάρις'),
   (0.0324986, 'δόξα'),
   (0.025417708, 'χριστός_ἰησοῦς'),
   (0.024591021, 'πνεῦμα'),
   (0.023555052, 'θεός'),
   (0.023177946, 'κύριος_ἰησοῦς'),
   (0.01967139, 'ομιλια'),
   (0.016726203, 'πνεῦμα'),
   (0.016204532, 'δύναμις'),
   (0.0